# Szacowanie liczby subtasków do sprawdzenia

*będę wdzięczny za uwagi, weryfikcję i poprawki - [jacek@golem.network](jacek@golem.network)*

## Wstęp

Bardzo prosty model biorący pod uwagę ceny zadań w sieci Golemowej daje niezwykle zaskakujące rezultaty.
Okazuje się, że przy stałym prawdopodobieństwie oszukiwania przez providera i stałych kosztach obliczeń, marży 
i kary za złe wykonanie zadania, liczba subtasków do weryfikacji **jest niezależna od rozmiaru zadania i bardzo mała (mniejsza niż 10, a zwykle mniejsza niż 3)**

Intuicja, jaka za tym stoi, jest dość jasna. W sekcji **Kilka przypadków** jest przestawione rozwinięcie tego tematu (pierwszy podpunkt, $s=1, R_{-}=R_{+}=0$.  
Pierwsza obserwacja jest taka, że providerowi nigdy nie opłaca się zawsze oszukiwać. (oczywiste, że gdyby zawsze to robił, a my sprawdzamy chodziaż jeden subtask, to na pewno zostanie złapany i nic nie zarobi, a straci czas na samo uruchomienie zadania, transfer danych itp).  
Załóżmy więc, że provider policzył już jeden subtask. Jeśli prowizja jest mała, to pomińmy ją przy ocenie kosztów - wtedy wartość pracy, którą wykonał provider, jest równa $\frac{1}{M}$ wartości zadania, gdzie $M$ to liczba wszystkich subtasków. Jeśli pozostałe policzy źle, a my sprawdzamy dla uproszczenia tylko jeden subtask, to ma szasę wygrać z prawdodpodobieństwem $\frac{1}{M}$ zapłatę $1 - \frac{1}{M}$ (zapłata wynosi $1$, ale provider wydał już część na obliczenie części zadania) i z prawdodpodobieństwem $\frac{M-1}{M}$ przegrać $\frac{1}{M}$ (właśnie te już zainwestowane zasoby). Widać, że rzeczy się upraszczają - i wychodzi na to, że wartość oczekiwana jest nadal $0$.  
To rozumowanie można oczywiście zastosować do większej liczby obliczonych uczciwie subtasków.  
Czyli podsumowując, intuicja jest taka, że licząc uczciwie provider też inwestuje, i jego ryzyko równoważy się z ryzykiem requestora - stąd bierze się stała liczba wymaganych do przetestowania subtasków.

## Model

### Oznaczenia

$M$ - liczba subtasków  
$l$ - prawdopodobieństwo, że provider będzie oszukiwał  
$s$ - liczba subtasków, które weryfikujemy  
$K$ - koszt zadania (koszt surowej mocy obliczeniowej, prądu itp)  
$R_{+}$ - marża providera  
Czyli całkowita zapłata za zadanie to $K + R_{+}$.  
  
$R_{-}$ - kara za złe wykonanie zadania  
$\Pr = \Pr(s, l, M)$ - prawdopodobieństwo złapania nieuczciwego providera  
E - wartość oczekiwana kosztów, jakie ponosi provider (jeśli ujemne, to znaczy, że provider zarobił) 

### Wartość oczekiwana kosztów providera

$ E = \textrm{szansa bycia złapanym} * \textrm{koszta bycia złapanym} + (1 - \textrm{szansa bycia złapanym}) * \textrm{koszta nie bycia złapanym}$

Skoro provider zawsze wykonuje uczciwie $(1-l)*M$ subtasków, a koszt jednego $\frac{\textrm{koszt wszystkich zadań}}{\textrm{liczba wszystich zadań}} = \frac{K}{M}$, to zawsze (tzn niezależnie, czy zostanie złapany, czy nie) ponosi koszt $(1-l)*M * \frac{K}{M} = (1-l)*K$

**Koszt bycia złapanym**  
Jeśli provider zostanie złapany, to ponosi koszt obliczeń, które wykonał uczciwie, plus płaci karę za złe wykonanie zadania, czyli w sumie $(1-l)K + R_{-}$

** Koszt nie bycia złapanym**  
Jeśli provider nie zostanie złapany, to ponosi koszt obliczeń, które wykonał uczciwie,
ale zarabia $\textrm{koszt zadania} + \textrm{marża providera} = K + R_{+}$,  
w sumie wychodzi więc $(1-l)K - (K + R_{+})$

Mamy więc, że  

$$E = \Pr((1-l)K + R_{-}) + (1-\Pr)((1-l)K - (K + R_{+})) = \\
\Pr(1-l)K + \Pr R_{-} + (1-l)K - \Pr(1-l)K - (1-\Pr)(K + R_{+}) = \\
\Pr R_{-} + (1-l)K - (1-\Pr)K - (1-\Pr)R_{+} = \\
\Pr R_{-} - lK + \Pr K - (1-\Pr)R_{+} = \\
\Pr(R_{-} + \Pr R_{+} + \Pr K - R_{+} - lK = \\
\Pr(R_{-} + R_{+} + K) - R_{+} - lK $$

**Prawdopodobieństwo złapania**
Jeśli zadanie składa się z $M$ niezależnych subtasków, z czego $lM$ jest wykonanych nieuczciwie, a do złapania providera wystarczy znaleźć jedno nieuczciwie wykonane, to szansa na to jest równa  
$$\Pr = 1 - (1-l)^s$$  
w przypadku losowań z powtórzeniami lub
$$\Pr = 1 - \frac{(1-l)M \choose s}{M \choose s}$$  
w przypadku losowań bez powtórzeń.  
Przypadek z powtórzeniami jest pesymistycznym oszacowaniem przypadku bez powtórzeń, a dla dużych $M$ i małych $l$ dobrze go przybliża, a ułatwia zorientowanie się w równaniu, dlatego dalej będę używał pierwszego wzoru.

Wstawiając do równania na $E$ dostajemy
$$E = (1 - (1 - l)^s)(R_{-} + R_{+} + K) - R_{+} - lK$$

**Jak widać, rozwiązanie nie zależy od M** - to oznacza, że liczba subtasków, które musimy spradzić, jest stała i nie rośnie z rozmiarem zadania (z pewną uwagą na końcu).

## Kilka przypadków

Teraz sprawdźmy, co się dzieje w kilku przypadkach:  
 - Jeśli nie ma kar ani nagród (tzn marża wynosi $0$), to z $s=1$ mamy
 $$ E = lK - lK = 0 $$
 Oznacza to, że przy sprawdzeniu jednego subtaska, provider tak samo wychodzi na oszukiwaniu (w dowolnym stopniu), jak i na byciu uczciwym.
 - J.w., tylko $s=2$ mamy
 $$ E = (1 - (1 - 2l + l^2))K - lK = lK - l^2K = l(1 - l)K $$
 Oznacza to, że przy sprawdzeniu dwóch subtasków provider najlepiej wychodzi na byciu cały czas uczciwym lub cały czas nieuczciwym - $l = 0$ lub $l = 1$ (bo nie ma kar i nic go to nie kosztuje)
 - Jeśli kara i marża są duże (rzędu kosztu zadania), $s=2$
 $$ e = (1 - (1 - 2l + l^2))(3K) - K - lK = 6lK - 2l^2K - K < 0$$
 Oznacza to, że providerowi opłaca się w takiej sytuacji oszukiwać. Widać, że w tej sytuacji składnik $K$ dominuje, więc musielibyśmy sprawić, żeby $(1-l)^s$ było relatywnie małe, czyli $s$ rośnie nam asymptotycznie logarytmicznie szybko, ale z dużą stałą (zależną od $M$, jeśli $l$ jest rzędu $\frac{1}{M}$).

Jednak nie musimy za bardzo przejmować się przypadkiem 3, bo w praktyce okazuje się, że $s$ może być dość małe (rzędu **stałej 2**)  
Przykładowe dane:

In [18]:
l = 0.1
K = 100.
Rplus = 50.
Rminus = 200.
s = 2.

E = (1.-(1.-l)**s)*(K + Rplus + Rminus) - Rplus - l*K

In [19]:
print(E)
if E < 0:
    print("Providerowi opłaca się taka sytuacja")
elif E == 0:
    print("Provider robi co chce")
else:
    print("Providerowi nie opłaca się taka sytuacja")

6.499999999999986
Providerowi nie opłaca się taka sytuacja


# Praktyczne podejście

Jak duże trzeba w takim razie ustalić s?  
Załóżmy, że dolnym limitem zarobku, na jaki może skusić się provider, żeby modyfikować kod golema, wstrzykiwać swoje wyniki itp jest $5$% wartości zadania.  

Załóżmy dalej, że wartość zadania to $100$. Niech minimalna prowizja za zadanie wynosi $10$, maksymalna $100$, minimalna kara $100$, maksymalna kara $1000$.

In [25]:
# Wtedy pesymistyczny przypadek to gdy 
l = 0.05
Rminus = 100
Rplus = 100
K = 100

Czyli żeby $E < 0$, $s$ musi spełniać
$$
\begin{aligned}
E &< 0 \\
R_{+} + lK &< (1 - (1 - l)^s)(R_{-} + R_{+} + K) \\
\frac{R_{+} + lK}{(R_{-} + R_{+} + K)} &< (1 - (1 - l)^s)\\
(1 - l)^s &< - \frac{R_{+} + lK}{(R_{-} + R_{+} + K)} + 1 \\
s &> log_{1-l}[- \frac{R_{+} + lK}{(R_{-} + R_{+} + K)} + 1]
\end{aligned}
$$

In [31]:
# Podstawiając dostajemy
from math import log
print("s > {}".format(log(-(Rplus + l*K)/(Rminus + Rplus + K) + 1, 1-l)))

s > 8.398425588296972


**Czyli niezależnie od rozmiaru zadania, wystarczy sprawdzić $9$ subtasków!**

In [37]:
# Weźmy teraz przypadek średni, kiedy
l = 0.05
Rminus = 200
Rplus = 10
K = 100

In [38]:
# Podstawiając dostajemy
from math import log
print("s > {}".format(log(-(Rplus + l*K)/(Rminus + Rplus + K) + 1, 1-l)))

s > 0.966928362304796


**Czyli nawet jedno sprawdzenie by wystarczyło.**

# Uwagi

 1. Ze wzoru $$E = (1 - (1 - l))^s(R_{-} + R_{+} + K) - R_{+} - lK$$ wynika, że niezależnie, jak duże dobierzemy $s$, provider zawsze jest w stanie znaleźć $l$ na tyle małe, żeby opłacało mu się oszukiwać (to nie jest do końca prawda, bo jeśli $s$ będzie rzędu $M$, to wzór na kombinacje bez powtórzeń przestaje się dobrze przybliżać wzorem na kombinacje bez powtórzeń).
 2. Cała analiza tutaj przeprowadzona ma przedstawić problem od strony ataku racjonalnego providera, tzn providera, który podejmuje działania, jeśli ich wartość oczekiwana jest mniejsza od zera.  
 To założenie nie musi być spełnione - w szczególności rozważając ataki na sieć, atakujący providerzy mogą poświęcać część zasobów w zamian za wypełnienie innych celów nie ujętych w modelu (np utrata zaufania do sieci, spadek wartości tokenu itp).  
 Obrona przed takimi "nieracjonalnymi" providerami może być bardzo trudna, z uwagi na to, że są w stanie operować bardzo celnie i małym kosztem.  
 Usecase: nieuczciwy provider zawsze renderuje obrazek z kilkoma wścieklezielonymi pikselami na środku. Zwykłe metody weryfikacji nie są w stanie tego wykryć (w terminologii niniejszego opracowania - $l$ jest bardzo małe), a skutecznie zniechęca to do używania Golema potencjalnych requestorów, zmniejsza zaufanie do systemu itp.  
 **Obrona przed takimi atakami musi być głębiej przemyślana**.